In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime

from data_helpers.syntax_graph import SyntaxGraph
from data_helpers.nc21_reader import Nc21Reader

# functions for creating database and collecting collocations
from collect_functions_1 import *

# Tabel 1 andmete kogumine

In [3]:
%%time
# takes ca 10 mins
file_name = 'data/nc19_Balanced_Corpus.vert'

my_nc21_reader = Nc21Reader(
   file_name = file_name
)

CPU times: user 50 µs, sys: 0 ns, total: 50 µs
Wall time: 53.2 µs


In [4]:
%%time

TYPE = 'obl_verb'
TABLENAME = f'{TYPE}'
BATCH_SIZE = 500000

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")
db_file_name = f"balanced_{TYPE}_collocations_{date_time}.db"

my_sqlite_db = DbMethods(db_file_name=db_file_name, table1_name=TYPE, table2_name=TYPE+'_examples')
my_sqlite_db.prep_coll_db()


# hoitakse algusest lõpuni mälus
verb_global_stat = {}

# kollokatsioonid, tühjendatakse peale igat salvestamist
collocations = {}
count = 0
for collection_id, graph in my_nc21_reader.get_sentences():
    
    count += 1
   
    collocations, verb_global_stat, = extract_something(graph, collection_id, collocations, verb_global_stat )
    
    if not collection_id == 0 and not collection_id % BATCH_SIZE:
        my_sqlite_db.save_coll_to_db(collocations, collection_id)
        collocations = {}
    
    
# saving last batch
my_sqlite_db.save_coll_to_db(collocations, collection_id)
collocations = {}
my_sqlite_db.index_fields()


df = verb_stat_to_df(verb_global_stat, f'balanced_verb_kids_{date_time}.csv')
df.to_sql(name='verbs_global_stat', con=my_sqlite_db._connection)

data/nc19_Balanced_Corpus.vert


TSV lines:  66%|██████▌   | 9853063/15006094 [03:14<15:10, 5656.95it/s] 

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 500000


TSV lines: 100%|██████████| 15006094/15006094 [04:45<00:00, 52549.70it/s]


andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 728148


NameError: name 'verb_stat_to_csv' is not defined